## Task: Restaurant Recommendation
###### Objective: Create a restaurant recommendation system based on user preferences.

 Steps:
 
 Preprocess the dataset by handling missing values and encoding categorical variables.
 
 Determine the criteria for restaurant recommendations (e.g., cuisine preference, price range).
 
 Implement a content-based filtering approach where users are recommended restaurants similar to their preferred criteria.
 
 Test the recommendation system by providing sample user preferences and evaluating the quality of recommendations.


In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the dataset
df = pd.read_csv('dataset.csv')
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


# Preprocessing

## Handle missing values

In [3]:

df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [4]:
# Separate the data into rows with known Cuisines and unknown Cuisines
data= df # making a copy of the original data set
known_cuisines = data[data['Cuisines'].notna()]
unknown_cuisines = data[data['Cuisines'].isna()]

In [5]:
unknown_cuisines

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
84,17284105,Cookie Shoppe,216,Albany,"115 N Jackson St, Albany, GA 31701",Albany,"Albany, Albany",-84.154000,31.577200,NaN,...,Dollar($),No,No,No,No,1,3.4,Orange,Average,34
87,17284211,Pearly's Famous Country Cookng,216,Albany,"814 N Slappey Blvd, Albany, GA 31701",Albany,"Albany, Albany",-84.175900,31.588200,NaN,...,Dollar($),No,No,No,No,1,3.4,Orange,Average,36
94,17284158,Jimmie's Hot Dogs,216,Albany,"204 S Jackson St, Albany, GA 31701",Albany,"Albany, Albany",-84.153400,31.575100,NaN,...,Dollar($),No,No,No,No,1,3.9,Yellow,Good,160
297,17374552,Corkscrew Cafe,216,Gainesville,"51 W Main St, Dahlonega, GA 30533",Dahlonega,"Dahlonega, Gainesville",-83.985800,34.531800,NaN,...,Dollar($),No,No,No,No,3,3.9,Yellow,Good,209
328,17501439,Dovetail,216,Macon,"543 Cherry St, Macon, GA 31201",Macon,"Macon, Macon",-83.627979,32.836410,NaN,...,Dollar($),No,No,No,No,3,3.8,Yellow,Good,102
346,17606621,HI Lite Bar & Lounge,216,Miller,"109 N Broadway Ave, Miller, SD 57362",Miller,"Miller, Miller",-98.989100,44.515800,NaN,...,Dollar($),No,No,No,No,1,3.4,Orange,Average,11
368,17059060,Hillstone,216,Orlando,"215 South Orlando Avenue, Winter Park, FL 32789",Winter Park,"Winter Park, Orlando",-81.365260,28.596682,NaN,...,Dollar($),No,No,No,No,3,4.4,Green,Very Good,1158
418,17142698,Leonard's Bakery,216,Rest of Hawaii,"933 Kapahulu Ave, Honolulu, HI 96816",Kaimuki,"Kaimuki, Rest of Hawaii",-157.813432,21.284586,NaN,...,Dollar($),No,No,No,No,1,4.7,Dark Green,Excellent,707
455,17616465,Tybee Island Social Club,216,Savannah,"1311 Butler Ave, Tybee Island, GA 31328",Tybee Island,"Tybee Island, Savannah",-80.848297,31.995810,NaN,...,Dollar($),No,No,No,No,1,3.9,Yellow,Good,309


In [6]:
# Assuming 'df' is your DataFrame
cuisines_in_albany = df[df['City'] == 'Savannah'].sort_values(by='Votes', ascending=False)

# Print the sorted cuisines in Albany
print(cuisines_in_albany['Cuisines'])


452               American, Seafood, Southern
436                                  Southern
439                        American, Southern
437                                    Burger
454                                   Seafood
438             Desserts, Sandwich, Ice Cream
447                          Breakfast, Cajun
440    International, Mediterranean, Sandwich
441              American, Bar Food, Sandwich
451                  Cafe, Sandwich, Southern
448                Breakfast, Diner, Sandwich
444                        American, Bar Food
450                     Pizza, Seafood, Steak
443                       American, Breakfast
445                     Breakfast, Vegetarian
442                  American, Seafood, Steak
453                                  American
449                  Coffee and Tea, Desserts
455                                       NaN
446                       American, Breakfast
Name: Cuisines, dtype: object


In [7]:
# missing at Albany
df.loc[df['Restaurant Name'] == 'Cookie Shoppe', 'Cuisines'] = 'Coffee, Tea, cookie'
df.loc[df['Restaurant Name'] == "Pearly's Famous Country Cookng", 'Cuisines'] = 'American, Breakfast, Diner'
df.loc[df['Restaurant Name'] == "Jimmie's Hot Dogs", 'Cuisines'] = 'Hot Dogs'

In [8]:
df.loc[df['Restaurant Name'] == "Corkscrew Cafe", 'Cuisines'] = 'Coffee and Tea, Sandwich'
df.loc[df['Restaurant Name'] == 'Dovetail', 'Cuisines'] = 'Italian'
df.loc[df['Restaurant Name'] == 'HI Lite Bar & Lounge', 'Cuisines'] = 'American, Breakfast, Diner'
df.loc[df['Restaurant Name'] == 'Dovetail', 'Cuisines'] = 'Italian'
df.loc[df['Restaurant Name'] == 'Hillstone', 'Cuisines'] = 'American, BBQ, Sandwich'
df.loc[df['Restaurant Name'] == "Leonard's Bakery", 'Cuisines'] = 'Breakfast, Burger'
df.loc[df['Restaurant Name'] == 'Tybee Island Social Club', 'Cuisines'] = 'American, Seafood, Southern'


In [9]:

df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

### Fixing Restaurant Names

In [9]:
restaurant_names = df['Restaurant Name']
# List out the strings that contain "���", "��", or "�"
error_restaurant_names = [name for name in restaurant_names if "�" in name]

# Print the filtered list
print("Error Restaurant Names: \n", error_restaurant_names)


Error Restaurant Names: 
 ['Caf�� Daniel Briand', 'Pizza �� Bessa', 'Sandubas Caf��', 'Tayp��', 'Manzu��', 'Braseiro da G��vea', 'Zaz�� Bistr�� Tropical', 'Fil�� de Ouro', 'Apraz�_vel', 'Terra�_o It��lia', 'Divino Fog��o', 'Esquina Mocot�_', 'Cev�_che Tapas Bar & Restaurant', 'Caf�� Tu Tu Tango', "Hollerbach's Willow Tree Caf��", 'MoMo Caf�� - Courtyard By Marriott', 'Caf�� Bogchi', 'Caf�� Kitchen', 'Gallery Caf�� - Hyatt Place', "Longitude 77��03' Bar - Le Meridien Gurgaon", 'bu��no', 'Queens Caf��', 'Caf�� Gramophone', 'M Cr��me', 'West��ross', 'bu��no', "Chawla's�_", "Chawla's�_", 'Caf�� Burger BC', 'The Basement Caf��', 'Caf�� Riverrun', 'Con�_u', 'Sahib��s Barbeque by Ohri��s', 'Hobnob Gourmet Caf��bar', 'Chemistry Caf��', 'NESCAF� Illusions', "Chawla's�_", 'It��s Sinful', 'Caf�� MRP', 'D�_ner Grill', 'Caf�� Healthilicious', 'D�_ner Grill', 'LaBont��', 'Chhalava - �__Lava', 'The Walled City - Caf�� & Lounge', 'Bon App��tit', "Chawla's�_", 'Cottage Caf�� by Smoothie factory', 'Dial

In [10]:

# Replace incorrect names with correct ones
df['Restaurant Name'] = df['Restaurant Name'].replace({
   'Caf� Daniel Briand': 'Café Daniel Briand','Caf�� Daniel Briand':'Café Daniel Briand',
   'Pizza � Bessa': 'Pizza à Bessa','Pizza �� Bessa':'Pizza à Bessa',
   'Sandubas Caf�': 'Sandubas Café','Sandubas Caf��':'Sandubas Café',
   'Tayp�': 'Taypé','Tayp��': 'Taypé',
   'Manzu�': 'Manzú','Manzu��': 'Manzú',
   'Braseiro da G�vea': 'Braseiro da Gávea','Braseiro da G��vea': 'Braseiro da Gávea',
   'Zaz� Bistr� Tropical': 'Zazá Bistrô Tropical','Zaz�� Bistr�� Tropical': 'Zazá Bistrô Tropical',
   'Fil� de Ouro': 'Filé de Ouro','Fil�� de Ouro': 'Filé de Ouro',
   'Apraz�_vel': 'Aprazível',
   'Terra�_o It�lia': 'Terraço Itália','Terra�_o It��lia': 'Terraço Itália',
   'Divino Fog�o': 'Divino Fogão','Divino Fog��o': 'Divino Fogão',
   'Esquina Mocot�_': 'Esquina Mocotó',
   'Cev�_che Tapas Bar & Restaurant': 'Ceviché Tapas Bar & Restaurant',
   'Caf� Tu Tu Tango': 'Café Tu Tu Tango',   'Caf�� Tu Tu Tango': 'Café Tu Tu Tango',
   "Hollerbach's Willow Tree Caf�": "Hollerbach's Willow Tree Café",   "Hollerbach's Willow Tree Caf��": "Hollerbach's Willow Tree Café",
   'MoMo Caf� - Courtyard By Marriott': 'MoMo Café - Courtyard By Marriott',
       'MoMo Caf�� - Courtyard By Marriott': 'MoMo Café - Courtyard By Marriott',
   'Caf� Bogchi': 'Café Bogchi',   'Caf�� Bogchi': 'Café Bogchi',
    'Caf� Kitchen': 'Café Kitchen','Caf�� Kitchen': 'Café Kitchen',
   'Gallery Caf� - Hyatt Place': 'Gallery Café - Hyatt Place',   'Gallery Caf�� - Hyatt Place': 'Gallery Café - Hyatt Place',
   "Longitude 77��03' Bar - Le Meridien Gurgaon": "Longitude 77°03' Bar - Le Meridien Gurgaon",
   'bu�no': 'buñno',
   'Queens Caf�': 'Queens Café','Queens Caf��': 'Queens Café',
   'Caf� Gramophone': 'Café Gramophone','Caf�� Gramophone': 'Café Gramophone',
   'M Cr�me': 'M Crème','M Cr��me': 'M Crème',
   'West�ross': 'Westcross','West��ross': 'Westcross',
   'bu�no': 'buñno','bu��no': 'buñno',
   "Chawla's�_": "Chawla's",
   "Chawla's�_": "Chawla's",
   'Caf� Burger BC': 'Café Burger BC','Caf�� Burger BC': 'Café Burger BC',
   'The Basement Caf�': 'The Basement Café','The Basement Caf��': 'The Basement Café',
   'Caf� Riverrun': 'Café Riverrun','Caf�� Riverrun': 'Café Riverrun',
   'Con�_u': 'Coñu',
   'Sahib�s Barbeque by Ohri�s': "Sahib's Barbeque by Ohri's",'Sahib��s Barbeque by Ohri��s': "Sahib's Barbeque by Ohri's",
   'Hobnob Gourmet Caf�bar': 'Hobnob Gourmet Cafébar','Hobnob Gourmet Caf��bar': 'Hobnob Gourmet Cafébar',
   'Chemistry Caf�': 'Chemistry Café','Chemistry Caf��': 'Chemistry Café',
   'NESCAF� Illusions': 'NESCAFÉ Illusions',
   "Chawla's�_": "Chawla's",
   'It�s Sinful': "It's Sinful", 'It��s Sinful': "It's Sinful",
   'Caf� MRP': 'Café MRP','Caf�� MRP': 'Café MRP',
   'D�_ner Grill': 'Dîner Grill',
   'Caf� Healthilicious': 'Café Healthilicious',
   'D�_ner Grill': 'Dîner Grill',
   'LaBont�': 'LaBonté',
   'Chhalava - �__Lava': 'Chhalava - Lava',
   'The Walled City - Caf� & Lounge': 'The Walled City - Café & Lounge',
   'Bon App�tit': 'Bon Appétit',
   "Chawla's�_": "Chawla's",
   'Cottage Caf� by Smoothie factory': 'Cottage Café by Smoothie factory',
   'Dialogue Lounge & Caf�': 'Dialogue Lounge & Café',
   "Ping's Caf� Orient": "Ping's Café Orient",
   "Chawla's�_": "Chawla's",
   'Delhite P�tisserie': 'Delhite Pâtisserie',
   'Arabian & Turkish Caf�': 'Arabian & Turkish Café',
   'D�_ner Grill': 'Dîner Grill',
   'Die B�ckerei': 'Die Bäckerei',
   'High Street Caf�': 'High Street Café',
   'The Junkyard Caf�': 'The Junkyard Café',
   'AMPM Caf� & Bar': 'AMPM Café & Bar',
   'Caf� Foreground': 'Café Foreground',
   "Chawla's�_": "Chawla's",
   'Rosart� Chocolate': 'Rosarté Chocolate',
   'Caf� Befikre': 'Café Befikre',
   'Caf� 101': 'Café 101',   'Caf�� 101': 'Café 101',
   'Tin Town Caf�': 'Tin Town Café',
   'Superstar Caf�': 'Superstar Café',
   'H�_agen-Dazs': 'Häagen-Dazs',
   'The Fashion Street Caf�': 'The Fashion Street Café',
   'Hearken Caf�': 'Hearken Café',
   'Remember Me Caf�': 'Remember Me Café',
   '4 Barrels Caf� & Lounge': '4 Barrels Café & Lounge',
   'Caf� Knosh - The Leela Ambience Convention Hotel': 'Café Knosh - The Leela Ambience Convention Hotel',
       'Caf�� Knosh - The Leela Ambience Convention Hotel': 'Café Knosh - The Leela Ambience Convention Hotel',
   'The Village Caf�': 'The Village Café','The Village Caf��': 'The Village Café',
   'Phonebooth Caf�': 'Phonebooth Café','Phonebooth Caf��': 'Phonebooth Café',
   'Caf� Doo Ghoont': 'Café Doo Ghoont','Caf�� Doo Ghoont': 'Café Doo Ghoont',
   'TBH �� To Be Healthy': 'TBH – To Be Healthy',
   'They�_����': 'They',
   'Caff� La Poya': 'Caffè La Poya',
   'More Than Caf�': 'More Than Café','More Than Caf��': 'More Than Café',
   'Caff� La Poya': 'Caffè La Poya','Caff�� La Poya': 'Caffè La Poya',
   'Elixir Health Caf�': 'Elixir Health Café','Elixir Health Caf��': 'Elixir Health Café',
   '#Urban Caf�': '#Urban Café','#Urban Caf��': '#Urban Café',
   'The Chickmunks Caf�': 'The Chickmunks Café', 'The Chickmunks Caf��': 'The Chickmunks Café',
   "Chawla's�_": "Chawla's",
   'KBC�_': 'KBC',
   "Chef's Basket Pop Up Caf�": "Chef's Basket Pop Up Café","Chef's Basket Pop Up Caf��":"Chef's Basket Pop Up Café",
   'Saut�ed Stories': 'Sautéed Stories','Saut��ed Stories': 'Sautéed Stories',
   'Freshco - The Health Caf�': 'Freshco - The Health Café',   'Freshco - The Health Caf��': 'Freshco - The Health Café',
   'Eden Noodles Cafe �__·�_��_��_��': 'Eden Noodles Cafe','Eden Noodles Cafe �__·�_��_��_��':'Eden Noodles Cafe',
   'Grand Caf� & Beach': 'Grand Café & Beach',    'Grand Caf�� & Beach': 'Grand Café & Beach',
   'Masaba��۱ Kebap�_۱s۱': 'Masabaşı Kebapçısı',
   'Me��hur Tavac۱ Recep Usta': 'Meşhur Tavacı Recep Usta',
   '�ukura��a Sofras۱': 'Çukurağa Sofrasɪ',
   'Me��hur �_z�_elik Aspava':'Meşhur Özçelik Aspava',
    'Masaba��۱':'Masabaşı',
    'D�_vero��lu':'Döveroğlu', 
    'Pizza ��l Forno':'Pizza ël Forno', 
    'Emirgan S�_ti��':'Emirgan Sütüş', 
    'Leman K�_lt�_r':'Leman Kültür', 
    'Dem Karak�_y':'Dem Karaköy', 
    'Karak�_y G�_ll�_o��lu':'Karaköy Güllüoğlu', 
    'Ceviz A��ac۱':'Ceviz Ağacɪ', 
    'A���k Kahve':'Açık Kahve',
    'Caf�� Foreground': 'Café Foreground',
    'Rosart�� Chocolate':'Rosart Chocolate', 
    'Caf�� Befikre':'Café Befikre', 
    'Tin Town Caf��':'Tin Town Café', 
    'Superstar Caf��':'Superstar Café', 
    'The Fashion Street Caf��':'The Fashion Street Café', 
    'Hearken Caf��':'Hearken Café', 
    'Remember Me Caf��':'Remember Me Café', 
    '4 Barrels Caf�� & Lounge':'4 Barrels Café & Lounge',
})

#### Fixing City Names

In [11]:

city_names = df['City']
# List out the strings that contain "���", "��", or "�"
error_city_names = [name for name in city_names if "�" in name]

# Print the filtered list
print("Error Restaurant Names: \n", error_city_names)


Error Restaurant Names: 
 ['Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'Bras�_lia', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', 'S��o Paulo', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul', '��stanbul']


In [12]:
df['City'] = df['City'].replace({
    'Bras�_lia':'Brasília',
    'S��o Paulo':'São Paulo',
    '��stanbul': 'İstanbul'
})

In [13]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,İstanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz Ağacɪ,208,İstanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,İstanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,Açık Kahve,208,İstanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


## step3 :

In [14]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

# Encoding 'Cuisines' (We might need to split the cuisines and then encode)
df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(', '))

# Encoding the 'City' and 'Locality' columns
label_encoder_city = LabelEncoder()
df['Encoded City'] = label_encoder_city.fit_transform(df['City'])
city_mapping = dict(zip(df['Encoded City'], df['City']))

label_encoder_locality = LabelEncoder()
df['Locality'] = label_encoder_locality.fit_transform(df['Locality'])

# Convert other categorical variables to numerical
df['Has Table booking'] = df['Has Table booking'].map({'Yes': 1, 'No': 0})
df['Has Online delivery'] = df['Has Online delivery'].map({'Yes': 1, 'No': 0})
df['Is delivering now'] = df['Is delivering now'].map({'Yes': 1, 'No': 0})
df['Switch to order menu'] = df['Switch to order menu'].map({'Yes': 1, 'No': 0})

# Check the dataframe after encoding
df.head()


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Encoded City
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",171,"Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"[French, Japanese, Desserts]",...,1,0,0,0,3,4.8,Dark Green,Excellent,314,73
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",593,"Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,[Japanese],...,1,0,0,0,3,4.5,Dark Green,Excellent,591,73
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...",308,"Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"[Seafood, Asian, Filipino, Indian]",...,1,0,0,0,4,4.4,Green,Very Good,270,75
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...",862,"SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"[Japanese, Sushi]",...,0,0,0,0,4,4.9,Dark Green,Excellent,365,75
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...",862,"SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"[Japanese, Korean]",...,1,0,0,0,4,4.8,Dark Green,Excellent,229,75


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Create a 'features' column for similarity comparison
df['features'] = df['Cuisines'].apply(lambda x: ' '.join(x)) + ' ' + df['Price range'].astype(str) + ' ' + df['Rating text']

# Vectorize the 'features' column
vectorizer = CountVectorizer().fit_transform(df['features'])
vectors = vectorizer.toarray()

In [16]:
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(vectors)

### Dash Board

In [17]:
# Function to get recommendations based on city, cuisine, and rating
def get_recommendations(city, cuisine, rating, cosine_sim=cosine_sim):
    encoded_city = label_encoder_city.transform([city])[0]
    # Filter the dataframe based on the input parameters
    filtered_df = df[(df['Encoded City'] == encoded_city) & (df['Cuisines'].apply(lambda x: cuisine in x)) & (df['Aggregate rating'] >= rating)]
    
    if filtered_df.empty:
        return pd.DataFrame(columns=['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Rating text'])
    
    # Get the indices of the filtered restaurants
    indices = filtered_df.index
    
    # Calculate the mean cosine similarity scores for the filtered restaurants
    sim_scores = cosine_sim[indices].mean(axis=0)
    
    # Sort the restaurants based on the similarity scores
    sim_scores = sorted(list(enumerate(sim_scores)), key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar restaurants
    sim_scores = sim_scores[1:11]
    
    # Get the restaurant indices
    restaurant_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar restaurants
    return df.iloc[restaurant_indices][['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Rating text']]

# Test the recommendation system with a sample restaurant
get_recommendations('İstanbul','Asian', 2)


,Restaurant Name,Cuisines,Price range,Aggregate rating,Rating text


In [ ]:
!pip install dash_bootstrap_components

In [18]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, Input, Output

In [19]:

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Restaurant Recommendation System"), width={"size": 6, "offset": 3})),
    dbc.Row(dbc.Col(
        dcc.Dropdown(
            id='city',
            options=[{'label': name, 'value': name} for name in df['City'].unique()],
            placeholder='Select a city',
            style={'width': '100%'}
        ), width={"size": 6, "offset": 3}
    )),
    dbc.Row(dbc.Col(
        dcc.Dropdown(
            id='cuisine',
            options=[{'label': cuisine, 'value': cuisine} for cuisine in df['Cuisines'].explode().unique()],
            placeholder='Select a cuisine',
            style={'width': '100%'}
        ), width={"size": 6, "offset": 3}
    )),
    dbc.Row(dbc.Col(
        dcc.Slider(
            id='rating',
            min=0,
            max=5,
            step=0.1,
            value=4,
            marks={i: str(i) for i in range(6)},
            tooltip={"placement": "bottom", "always_visible": True},
        ), width={"size": 6, "offset": 3}
    )),
    dbc.Row(dbc.Col(html.Button('Get Recommendations', id='submit-button', n_clicks=0), width={"size": 6, "offset": 3})),
    dbc.Row(dbc.Col(html.Div(id='recommendations-output'), width={"size": 6, "offset": 3}))
])

# Define the callback to update recommendations
@app.callback(
    Output('recommendations-output', 'children'),
    [Input('submit-button', 'n_clicks')],
    [Input('city', 'value'), Input('cuisine', 'value'), Input('rating', 'value')]
)
def update_recommendations(n_clicks, city, cuisine, rating):
    if n_clicks > 0 and city and cuisine and rating is not None:
        try:
            recommendations = get_recommendations(city, cuisine, rating)
            return html.Ul([html.Li(f"{row['Restaurant Name']} - Cuisines: {', '.join(row['Cuisines'])}, Price Range: {row['Price range']}, Rating: {row['Aggregate rating']}, {row['Rating text']}") for index, row in recommendations.iterrows()])
        except IndexError:
            return html.Div("No recommendations found. Please try different parameters.")
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)